In [6]:
from imports import *
from PreProcessor import *

_PreProcessor = PreProcessor()


In [7]:
function_names_train,function_segments_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_CC",'CC','train')
descriptions_train, summaries_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_UC",'UC','train')

function_names_test,function_segments_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_CC",'CC','test')
descriptions_test, summaries_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_UC",'UC','test')

In [8]:
_PreProcessor.setUpUnknown(function_names_train,function_segments_train,'CC')
_PreProcessor.setUpUnknown(descriptions_train,summaries_train,'UC')

_PreProcessor.setUpUnknown(function_names_test,function_segments_test,'CC')
_PreProcessor.setUpUnknown(descriptions_test,summaries_test,'UC')

In [20]:
with open('./pickles/DeepLearning/function_names_train.pkl', 'wb') as f:
       pickle.dump(function_names_train, f)
with open('./pickles/DeepLearning/function_segments_train.pkl', 'wb') as f:
        pickle.dump(function_segments_train, f)
with open('./pickles/DeepLearning/descriptions_train.pkl', 'wb') as f:
        pickle.dump(descriptions_train, f)
with open('./pickles/DeepLearning/summaries_train.pkl', 'wb') as f:
        pickle.dump(summaries_train, f)

with open('./pickles/DeepLearning/Vocab.pkl', 'wb') as f:
        pickle.dump(_PreProcessor.Vocab, f)

with open('./pickles/DeepLearning/function_names_test.pkl', 'wb') as f:
       pickle.dump(function_names_test, f)
with open('./pickles/DeepLearning/function_segments_test.pkl', 'wb') as f:
        pickle.dump(function_segments_test, f)
with open('./pickles/DeepLearning/descriptions_test.pkl', 'wb') as f:
        pickle.dump(descriptions_test, f)
with open('./pickles/DeepLearning/summaries_test.pkl', 'wb') as f:
        pickle.dump(summaries_test, f)

In [21]:
function_names_train = np.load('./pickles/DeepLearning/function_names_train.pkl',allow_pickle=True)
function_segments_train = np.load('./pickles/DeepLearning/function_segments_train.pkl',allow_pickle=True)
descriptions_train = np.load('./pickles/DeepLearning/descriptions_train.pkl',allow_pickle=True)
summaries_train = np.load('./pickles/DeepLearning/summaries_train.pkl',allow_pickle=True)
vocab = np.load('./pickles/DeepLearning/Vocab.pkl',allow_pickle=True)
function_names_test = np.load('./pickles/DeepLearning/function_names_test.pkl',allow_pickle=True)
function_segments_test = np.load('./pickles/DeepLearning/function_segments_test.pkl',allow_pickle=True)
descriptions_test = np.load('./pickles/DeepLearning/descriptions_test.pkl',allow_pickle=True)
summaries_test = np.load('./pickles/DeepLearning/summaries_test.pkl',allow_pickle=True)

### Converting dataset to indices

In [9]:
_PreProcessor.vocabToIndex()
_PreProcessor.dataSetToIndex(function_names_train,function_segments_train,'CC')
_PreProcessor.dataSetToIndex(descriptions_train,summaries_train,'UC')

_PreProcessor.dataSetToIndex(function_names_test,function_segments_test,'CC')
_PreProcessor.dataSetToIndex(descriptions_test,summaries_test,'UC')

In [10]:
x_train, labels = _PreProcessor.setUpLabels(function_names_train,function_segments_train,descriptions_train,summaries_train)


In [ ]:
print(x_train[0],labels[0] )

In [11]:
class TracibilityLinkDataset(Dataset):

    def __init__(self, x_train, y_train):

        # # getting the max number of funtions in a file along all files
        # #getting the longest name of a function along all functions in all files
        # max_number_function = max(len(_) for _ in x_train_function_names)
        # max_function_name_len = max(len(function) for file in x_train_function_names for function in file)

        # #getting the longest segment of a function along all functions in all files
        # max_function_segment_len = max(len(function) for file in x_train_function_segments for function in file)

        # x_train_function_names_padded = [file + [[]] * (max_number_function - len(file)) for file in x_train_function_names]
        # x_train_function_names_padded = [function + [vocab_size+1] * (max_number_function - len(function)) for file in x_train_function_names_padded for function in file]
        
        # # convert it to a tensor to be used in model
        # self.tensor_x_train_function_names_padded = torch.tensor(x_train_function_names_padded)
        
        self.tensor_x_train = x_train
        self.tensor_y_train = torch.tensor(y_train)
        print(len(self.tensor_x_train),  self.tensor_y_train.shape)


    # len and get item are very important --> used by dataloader
    def __len__(self):
        return len(self.tensor_y_train)

    def __getitem__(self, idx):
        print(self.tensor_x_train[idx])
        return self.tensor_x_train[idx], self.tensor_y_train[idx]


In [15]:
dataset_train = TracibilityLinkDataset(x_train, labels)
torch.save(dataset_train,'./dataset/teiid_dataset/DeepLearningDataset.pt')

3239676 torch.Size([3239676])


In [16]:
dataset_train = torch.load('./dataset/teiid_dataset/DeepLearningDataset.pt')

In [17]:
class DLModel(nn.Module):

  def __init__(self,embedding_matrix : np.array, embedding_dim : tuple, classes: int = 2):
      super(DLModel, self).__init__()

      # first embedding layer: input size --> vocab size, output size --> feature vector of vocan size(one hot encoding), weigths --> one hot encoding
      self.one_hot_embedding = nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim = embedding_dim[1], _weight = torch.tensor(embedding_matrix))
      
      # self-attention layer --> num_heads = 1, query, key, value will all be the input
      self.attention_layer = nn.MultiheadAttention(embed_dim=embedding_dim[1], num_heads=1, batch_first = True)

      # softmax ?
      # self.linear = nn.Linear(2 * hidden_size_layer_chars, classes, bias=True)

  def forward(self, x_train: list):
      
    function_names, function_segments, descriptions, summaries = x_train
    function_names_embedding = self.one_hot_embedding(function_names)
    function_segments_embedding = self.one_hot_embedding(function_segments)
    descriptions_embedding  = self.one_hot_embedding(descriptions)
    summaries_embedding = self.one_hot_embedding(summaries)
    print(function_names_embedding.shape)
    print(function_segments_embedding.shape)
    print(descriptions_embedding.shape)
    print(summaries_embedding.shape)

      





    # feature_vector = self.embedding_words(sentences) # sentences * words *253

    # feature_vector = feature_vector.to(dtype=torch.float)
    # lstm_output = self.lstm_layer_words(feature_vector) # sentences * words *200
    # #print(lstm_output[0].shape)  
    # # looping over lstm_output to remove unwanted feature vectors
    # no_of_sentences, length_of_sentence, feature_vector_size = lstm_output[0].shape
    # lstm_output_words= lstm_output[0]
    # lstm_output_words_2d=lstm_output_words.reshape(-1,lstm_output_words.shape[2])
    # #print(lstm_output_words_2d.shape)
    # embedding_char_output_4d=self.embedding_char(words)  # sentences * words * chars * one hot encoding for each char
    # #words * chars * one hot encoding for each char
    # embedding_char_output_3d=embedding_char_output_4d.reshape(-1,embedding_char_output_4d.shape[2],embedding_char_output_4d.shape[3])
    # #print(embedding_char_output_3d.shape)

    # #print(lstm_output_words_2d[0]) # words*hot encoding vector
    # embedding_char_output_3d_concatenated=np.zeros((embedding_char_output_3d.shape[0],embedding_char_output_3d.shape[1],embedding_char_output_3d.shape[2]+lstm_output_words_2d.shape[1]))

    # # for word in range(len(lstm_output_words_2d)):
    # #        for chars in range(len(embedding_char_output_3d[word])):
    # #print(lstm_output_words_2d.shape)
    # lstm_output_words_2d=lstm_output_words_2d.detach().numpy()

    # lstm_output_words_2d = np.repeat(lstm_output_words_2d, repeats=embedding_char_output_3d.shape[1], axis=0)
    # #print(lstm_output_words_2d.shape)

    # lstm_output_words_2d=lstm_output_words_2d.reshape(lstm_output_words_2d.shape[0]//embedding_char_output_3d.shape[1],embedding_char_output_3d.shape[1],lstm_output_words_2d.shape[1])
    # #print(lstm_output_words_2d.shape)

    # embedding_char_output_3d_concatenated = np.append(embedding_char_output_3d.detach().numpy(),lstm_output_words_2d,axis=2)  # Append along rows

    # embedding_char_output_3d_concatenated = torch.tensor(embedding_char_output_3d_concatenated)
    # #print(embedding_char_output_3d_concatenated.shape)

    # embedding_char_output_3d_concatenated = embedding_char_output_3d_concatenated.to(dtype=torch.float)
    # #lstm_output_intermediate = self.intermediate_lstm_layer(embedding_char_output_3d_concatenated)
    # #print(embedding_char_output_3d_concatenated.shape)
    # lstm_output_char = self.lstm_layer_char(embedding_char_output_3d_concatenated)
    # final_output = self.linear(lstm_output_char[0])  # words * 13 char * 15 classs

    return None

In [18]:
def train(model, train_dataset, batch_size=200, epochs=1, learning_rate=0.01):

    # (1) create the dataloader of the training set (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size)

    # (2) make the criterion cross entropy loss
    # criterion = torch.nn.CrossEntropyLoss() 

    # (3) create the optimizer (Adam)
    # optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

    for epoch_num in range(epochs):
        # total_acc_train = 0
        # total_loss_train = 0
        # n_samples=0

        for x_train, y_train in tqdm(train_dataloader):
            # (6) do the forward pass
            output = model.forward(x_train)
            break
            
            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            # label: batch_size * 253 * 13
            # output: 25300 word * 13 char * 15 
            # k2eny fket el array l words msh sentence of words

            # batch_loss = criterion(output.view(-1, output.size(2)), train_label.view(-1))

            # # (8) append the batch loss to the total_loss_train

            # total_loss_train += batch_loss
            
            # # (9) calculate the batch accuracy (just add the number of correct predictions)
            # _,predicted=torch.max(output,2)  #512 * 104  for every word in every sentence we choose one tag form the seventen tag 
            # acc=(predicted==train_label.view(-1, train_label.shape[2])).sum().item()
            # n_samples += train_label.size(0)*train_label.size(1)*train_label.size(2)
            # total_acc_train += acc
            # print(100*total_acc_train/n_samples)

            # # (10) zero your gradients
            # optimizer.zero_grad()

            # # (11) do the backward pass
            # batch_loss.backward()
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            # # (12) update the weights with your optimizer
            # optimizer.step()
        # epoch loss
        # epoch_loss = total_loss_train / len(train_dataset)

        # # (13) calculate the accuracy
        # epoch_acc =100*total_acc_train/n_samples

        # print(
        #     f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        #     | Train Accuracy: {epoch_acc}\n')
        break

  ##############################################################################################################

In [22]:
one_hot_embedding = np.eye(len(vocab.keys()) + 1)
model = DLModel(one_hot_embedding, one_hot_embedding.shape)

In [23]:
train(model, dataset_train)
# print(len(labels), len(x_train))

  0%|          | 0/16199 [00:00<?, ?it/s]

[[tensor([1]), tensor([1, 2]), tensor([3, 4]), tensor([1, 5]), tensor([6, 7, 8]), tensor([ 7,  9, 10, 11]), tensor([12, 13]), tensor([ 6, 14, 15, 13]), tensor([16, 17, 18]), tensor([19, 20, 21, 22]), tensor([23, 24]), tensor([25, 26, 18]), tensor([27, 28, 15, 13]), tensor([220,   5])], [tensor([29, 30, 31, 31, 29, 29, 30, 31, 32, 33, 34, 30, 31, 32,  1,  2, 35, 36,
        35, 33, 34,  1,  2, 35,  5, 36, 35,  5, 37,  5, 36,  5, 36,  5,  1,  2,
         2,  5, 38, 36, 39, 40, 41, 39, 40, 41, 39, 40, 41, 39, 29, 30, 31, 32,
        36,  2, 36,  5, 33, 34, 30, 31, 32, 33, 34, 37,  2, 36,  2, 37,  2, 36,
         5, 36,  2, 14, 13, 42]), tensor([ 1, 43,  5,  2, 35, 37,  5,  1,  5,  1,  5,  2,  5,  1,  5, 44, 45,  2,
         5,  3, 46, 47,  1,  2,  2,  5,  3, 48, 46, 37,  5, 49, 50,  1,  5,  1,
         5, 51, 52,  5, 53, 54, 55, 49, 50,  3, 56, 49, 50, 49, 50,  3, 56, 57,
         1,  2, 58, 56,  2,  5, 46,  5,  2,  5,  3, 56, 37,  5, 46,  1,  2, 46,
         5,  2,  5,  3, 59,  5, 60, 59

  0%|          | 0/16199 [00:01<?, ?it/s]

[[tensor([36]), tensor([  24, 1131]), tensor([220]), tensor([   3, 2456,  375,   18]), tensor([  7, 531]), tensor([220,  22]), tensor([220]), tensor([220, 289]), tensor([ 3, 59]), tensor([ 36, 345]), tensor([ 645, 1129,  201]), tensor([ 645, 1055,  201]), tensor([193]), tensor([  36, 1130,   22]), tensor([]), tensor([414, 193, 375])], [tensor([  79, 1083,   18,  405, 1083,   18,  405,    3, 1083,    3,   18,  716,
          77,  487,    3,  716,  340,  509,  509,  716,    3,   18,  509,  340,
         509,   34,  340,  509,   34,   22,   22,  716,    3,   22,   86,  220,
          22, 1083,   18,  340,  509,   34,  340,  509,   34,   79,   18,   36,
         345,  201,  220,   22,   18,   36,  345,    3,   18, 1085, 1091,  716,
           3,   18, 1087,  340,   79]), tensor([ 340,  509, 1131]), tensor([  22,    3, 1084,  340,  509,  509,   22,    3, 1084,  509,  509,   34,
         193,  375,  340,  509,  193,   79, 1083,   18, 1085,   77,   84,    3,
        1083,   18,   22, 1083,   

RuntimeError: each element in list of batch should be of equal size